In [1]:
from main import *

In [2]:
gw_no = 1

In [3]:
# see gameweek general info
r = requests.get('https://fantasy.premierleague.com/api/'+'bootstrap-static/').json()

In [4]:
r.keys()

dict_keys(['events', 'game_settings', 'phases', 'teams', 'total_players', 'elements', 'element_stats', 'element_types'])

In [5]:
gw_info = r['events'][gw_no - 1]

In [6]:
gw_info # want this kind of info for our leagues especially chip plays, most_selected, most_captained

{'id': 1,
 'name': 'Gameweek 1',
 'deadline_time': '2024-08-16T17:30:00Z',
 'release_time': None,
 'average_entry_score': 39,
 'finished': False,
 'data_checked': False,
 'highest_scoring_entry': 284112,
 'deadline_time_epoch': 1723829400,
 'deadline_time_game_offset': 0,
 'highest_score': 119,
 'is_previous': False,
 'is_current': True,
 'is_next': False,
 'cup_leagues_created': False,
 'h2h_ko_matches_created': False,
 'ranked_count': 8598588,
 'chip_plays': [{'chip_name': 'bboost', 'num_played': 144974},
  {'chip_name': '3xc', 'num_played': 221430}],
 'most_selected': 401,
 'most_transferred_in': 1,
 'top_element': 328,
 'top_element_info': {'id': 328, 'points': 14},
 'transfers_made': 0,
 'most_captained': 351,
 'most_vice_captained': 351}

In [7]:
league_name = 'rpk'  #'rpk', 'ifc', 'rbsc'
league_id = get_league_id(league_name)

# Pre match
- Players data
- Managers data
- Managers picks
    - captain selection
    - most transfers
    - selected by %

## Player data

In [8]:
# get latest player data, fun data to look at
players = get_dim_players()

In [9]:
players.columns

Index(['chance_of_playing_next_round', 'chance_of_playing_this_round', 'code',
       'cost_change_event', 'cost_change_event_fall', 'cost_change_start',
       'cost_change_start_fall', 'dreamteam_count', 'element_type', 'ep_next',
       'ep_this', 'event_points', 'first_name', 'form', 'id', 'in_dreamteam',
       'news', 'news_added', 'now_cost', 'photo', 'points_per_game',
       'second_name', 'selected_by_percent', 'special', 'squad_number',
       'status', 'team', 'team_code', 'total_points', 'transfers_in',
       'transfers_in_event', 'transfers_out', 'transfers_out_event',
       'value_form', 'value_season', 'web_name', 'minutes', 'goals_scored',
       'assists', 'clean_sheets', 'goals_conceded', 'own_goals',
       'penalties_saved', 'penalties_missed', 'yellow_cards', 'red_cards',
       'saves', 'bonus', 'bps', 'influence', 'creativity', 'threat',
       'ict_index', 'starts', 'expected_goals', 'expected_assists',
       'expected_goal_involvements', 'expected_goals_con

In [10]:
players.loc[players.id == 9, ['first_name', 'second_name']]

,first_name,second_name
8,Gabriel,Martinelli Silva


## Manager data

In [11]:
managers = get_dim_managers(league_id)

In [12]:
managers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 22 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   id                             8 non-null      int64 
 1   joined_time                    8 non-null      object
 2   started_event                  8 non-null      int64 
 3   favourite_team                 8 non-null      int64 
 4   player_first_name              8 non-null      object
 5   player_last_name               8 non-null      object
 6   player_region_id               8 non-null      int64 
 7   player_region_name             8 non-null      object
 8   player_region_iso_code_short   8 non-null      object
 9   player_region_iso_code_long    8 non-null      object
 10  years_active                   8 non-null      int64 
 11  summary_overall_points         8 non-null      int64 
 12  summary_overall_rank           8 non-null      int64 
 13  summary_e

## Manager picks

In [14]:
gw_picks = get_gw_picks(league_id, gw_no)

In [15]:
gw_picks

,element,position,multiplier,is_captain,is_vice_captain,manager_id
0,413,1,1,False,False,1576908
1,18,2,1,False,False,1576908
2,311,3,1,False,False,1576908
3,333,4,1,False,False,1576908
4,328,5,1,False,False,1576908
...,...,...,...,...,...,...
115,129,11,1,False,False,3607641
116,185,12,0,False,False,3607641
117,169,13,0,False,False,3607641
118,48,14,0,False,False,3607641


In [16]:
# join with other data for description
gw_picks = pd.merge(gw_picks, players, left_on='element', right_on='id') # get player names
gw_picks = pd.merge(gw_picks, managers, left_on='manager_id', right_on='id') # get manager names

### Captains

In [17]:
captains = gw_picks.loc[gw_picks.is_captain == True]


In [18]:
# who captain who?
captains.loc[
    :,
    ['first_name', 'second_name', 'player_first_name', 'player_last_name', 'name']
    ].sort_values('first_name')

,first_name,second_name,player_first_name,player_last_name,name
9,Alexander,Isak,Tawiwut,Charuwat,Chicks Casino CF
62,Alexander,Isak,Pattapong,Charoenchaipong,Morty FC
77,Alexander,Isak,sirawat,dulyavit,Sirawatd
94,Alexander,Isak,Atthapon,Parkart,Aekk72
39,Erling,Haaland,PHOOM T.,YENBAMROONG,PTHOMAS FC
111,Erling,Haaland,Nithiz,Onkaewmanee,OnkaewmaneeN.
16,Mohamed,Salah,Natthawat,Charoenkitmongkol,proove.club
46,Mohamed,Salah,Tachapon,Ratsameedara,TachaponR


In [19]:
# summary
captained_by = captains.groupby(['element', 'first_name', 'second_name']).size().sort_values(ascending=False).reset_index(name='managers_captained_by')
captained_by['percent_captained_by'] = round(captained_by.managers_captained_by / managers.shape[0] * 100, 2)

In [20]:
captained_by

,element,first_name,second_name,managers_captained_by,percent_captained_by
0,401,Alexander,Isak,4,50.0
1,328,Mohamed,Salah,2,25.0
2,351,Erling,Haaland,2,25.0


### most selected

In [21]:
selected_by = gw_picks[gw_picks.multiplier != 0].groupby(['element', 'first_name', 'second_name']).size().sort_values(ascending=False).reset_index(name='managers_selected_by')

In [22]:
selected_by['percent_selected_by'] = round(selected_by.managers_selected_by / managers.shape[0] * 100, 2)

In [23]:
selected_by.head(10)

,element,first_name,second_name,managers_selected_by,percent_selected_by
0,401,Alexander,Isak,7,87.5
1,17,Bukayo,Saka,6,75.0
2,311,Trent,Alexander-Arnold,6,75.0
3,351,Erling,Haaland,5,62.5
4,129,João Pedro,Junqueira de Jesus,5,62.5
5,199,Eberechi,Eze,5,62.5
6,201,Dean,Henderson,4,50.0
7,495,Pedro,Porro,4,50.0
8,317,Diogo,Teixeira da Silva,4,50.0
9,328,Mohamed,Salah,3,37.5


### transfers

In [24]:
# TODO

# Post match

In [25]:
# assert gw_info['finished']

In [26]:
points = get_gw_points(league_id, 1)

In [27]:
points

,manager_id,team_name,name,gw_no,points,transfers_cost,h2h_points,rank
0,1576908,Chicks Casino CF,Tawiwut Charuwat,1,87,0,87,1
1,4329910,proove.club,Natthawat Charoenkitmongkol,1,58,0,58,2
2,3611665,TachaponR,Tachapon Ratsameedara,1,51,0,51,3
3,2910243,Morty FC,Pattapong Charoenchaipong,1,47,0,47,4
4,7348222,Sirawatd,sirawat dulyavit,1,46,0,46,5
5,5377619,PTHOMAS FC,PHOOM T. YENBAMROONG,1,43,0,43,6
6,5961532,Aekk72,Atthapon Parkart,1,38,0,38,7
7,3607641,OnkaewmaneeN.,Nithiz Onkaewmanee,1,18,0,18,8


In [28]:
get_gw_weekly_pnl(points, league_name)

,manager_id,team_name,name,gw_no,points,transfers_cost,h2h_points,rank,is_unique_rank,pnl
0,1576908,Chicks Casino CF,Tawiwut Charuwat,1,87,0,87,1,True,300
1,4329910,proove.club,Natthawat Charoenkitmongkol,1,58,0,58,2,True,200
2,3611665,TachaponR,Tachapon Ratsameedara,1,51,0,51,3,True,100
3,2910243,Morty FC,Pattapong Charoenchaipong,1,47,0,47,4,True,0
4,7348222,Sirawatd,sirawat dulyavit,1,46,0,46,5,True,0
5,5377619,PTHOMAS FC,PHOOM T. YENBAMROONG,1,43,0,43,6,True,-100
6,5961532,Aekk72,Atthapon Parkart,1,38,0,38,7,True,-200
7,3607641,OnkaewmaneeN.,Nithiz Onkaewmanee,1,18,0,18,8,True,-300


In [29]:
# rpk only equal rank pnl hardcode
# points['pnl'] = []

# End of season

In [30]:
# df = pd.read_csv('./data/rbsc/fact_gw_rbsc.csv')
# for gw_no in range(37,38+1):
#     df = pd.concat([df, get_gw_weekly_pnl(get_gw_points(league_id, gw_no), league_name)], ignore_index=True)
#     gw_no += 1